In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r'C:\Users\jakep\OneDrive\Documents\CS639\citibike-network\2019_citibike.csv',
                parse_dates = ['Start Time'],
                usecols = ['Trip Duration', 'Start Time', 'Start Station ID', 'End Station ID'])
print(df)

          Trip Duration              Start Time  Start Station ID  \
0                   320 2019-01-01 00:01:47.401              3160   
1                   316 2019-01-01 00:04:43.736               519   
2                   591 2019-01-01 00:06:03.997              3171   
3                  2719 2019-01-01 00:07:03.545               504   
4                   303 2019-01-01 00:07:35.945               229   
...                 ...                     ...               ...   
20551692             93 2019-12-31 23:58:18.016              3141   
20551693            786 2019-12-31 23:58:59.896               490   
20551694            351 2019-12-31 23:59:03.695              3349   
20551695           1571 2019-12-31 23:59:21.361               252   
20551696            310 2019-12-31 23:59:55.296              3351   

          End Station ID  
0                 3283.0  
1                  518.0  
2                 3154.0  
3                 3709.0  
4                  503.0  
...      

In [3]:
stations = pd.read_csv(r'C:\Users\jakep\OneDrive\Documents\CS639\citibike-network\stations.csv')
stations = stations.rename(columns={'Unnamed: 0': 'id'})

### Determine which stations are in Manhattan

Using a shapefile of New York city boroughs, we can check whether a point lies within the polygon corresponding to Manhattan in the shapefile.

In [4]:
import shapefile
from shapely.geometry import shape, Point

r = shapefile.Reader(r"C:\Users\jakep\Downloads\Borough Boundaries\geo_export_43b35747-351a-4fbf-944b-eaee1f68385e.shp")
shapes = r.shapes()

polygon = shape(shapes[1]) # Manhattan has ID=1

def check(lon, lat):
    point = Point(lon, lat)
    return polygon.contains(point)

stations['Manhattan'] = stations.apply(lambda x: check(x['lon'], x['lat']), axis=1)
stations

,id,name,lon,lat,Manhattan
0,3160.0,Central Park West & W 76 St,-73.973747,40.778968,True
1,519.0,Pershing Square North,-73.977706,40.751873,True
2,3171.0,Amsterdam Ave & W 82 St,-73.976673,40.785247,True
3,504.0,1 Ave & E 16 St,-73.981656,40.732219,True
4,229.0,Great Jones St,-73.993790,40.727434,True
...,...,...,...,...,...
932,3740.0,W Broadway & W Houston St,-74.000472,40.726378,True
933,3884.0,Centre St & Seneca Ave,-73.902200,40.699690,False
934,3910.0,Greene Ave & Grandview Ave,-73.907856,40.709697,False
935,3909.0,Madison St & Woodward Ave,-73.904080,40.704080,False


In [8]:
manhattan_stations = stations[stations['Manhattan']].copy()
manhattan_stations_ids = set(manhattan_stations['id'].astype(int))

print(f'Number of Stations in Manhattan: {len(manhattan_stations_ids)}')

Number of Stations in Manhattan: 459


### Remove loops
We are not interested in rides that start and end at the same station.

In [10]:
same_station = df['Start Station ID'] == df['End Station ID']
print(f'Number of cyclical rides: {sum(same_station)}')
df = df[~same_station].copy()

Number of cyclical rides: 442977


### Determine ride times
Figure out whether each ride happened at day or night, and whether on a weekday or weekend.

In [11]:
df.loc[df['Start Time'].dt.dayofweek < 5, 'Weekday'] = 'Weekday'
df.loc[df['Start Time'].dt.dayofweek >= 5, 'Weekday'] = 'Weekend'

df.loc[(21 > df['Start Time'].dt.hour) & (df['Start Time'].dt.hour >= 7), 'Daytime'] = 'Day'
df.loc[(21 <= df['Start Time'].dt.hour) | (df['Start Time'].dt.hour < 7), 'Daytime'] = 'Night'

df

,Trip Duration,Start Time,Start Station ID,End Station ID,Weekday,Daytime
0,320,2019-01-01 00:01:47.401,3160,3283.0,Weekday,Night
1,316,2019-01-01 00:04:43.736,519,518.0,Weekday,Night
2,591,2019-01-01 00:06:03.997,3171,3154.0,Weekday,Night
3,2719,2019-01-01 00:07:03.545,504,3709.0,Weekday,Night
4,303,2019-01-01 00:07:35.945,229,503.0,Weekday,Night
...,...,...,...,...,...,...
20551692,93,2019-12-31 23:58:18.016,3141,3142.0,Weekday,Night
20551693,786,2019-12-31 23:58:59.896,490,513.0,Weekday,Night
20551694,351,2019-12-31 23:59:03.695,3349,3368.0,Weekday,Night
20551695,1571,2019-12-31 23:59:21.361,252,366.0,Weekday,Night


### Figure out which rides both started and ended in Manhattan

In [12]:
# Given a point(lon, lat) determine whether is it located in Manhattan
def station_in_city(start, stop):
    s = start in manhattan_stations_ids
    e = stop in manhattan_stations_ids
    return s and e
    
df['Manhattan'] = df.apply(lambda x: station_in_city(x['Start Station ID'], x['End Station ID']), axis = 1)

In [13]:
# Percentage of rides starting and ending in Manhattan
np.mean(df['Manhattan'])

0.768610732060519

In [18]:
# Only keep stations in Manhattan
df = df[df['Manhattan']].copy()
df

,Trip Duration,Start Time,Start Station ID,End Station ID,Weekday,Daytime,Manhattan
0,320,2019-01-01 00:01:47.401,3160,3283,Weekday,Night,True
1,316,2019-01-01 00:04:43.736,519,518,Weekday,Night,True
2,591,2019-01-01 00:06:03.997,3171,3154,Weekday,Night,True
3,2719,2019-01-01 00:07:03.545,504,3709,Weekday,Night,True
4,303,2019-01-01 00:07:35.945,229,503,Weekday,Night,True
...,...,...,...,...,...,...,...
20551690,122099,2019-12-31 23:55:30.726,3712,347,Weekday,Night,True
20551691,1838,2019-12-31 23:57:04.604,3711,515,Weekday,Night,True
20551692,93,2019-12-31 23:58:18.016,3141,3142,Weekday,Night,True
20551693,786,2019-12-31 23:58:59.896,490,513,Weekday,Night,True


In [19]:
df['End Station ID'] = df['End Station ID'].astype(int)

In [20]:
summary = df.groupby(['Start Station ID', 'End Station ID', 'Weekday', 'Daytime']).size()
summary = summary.unstack(fill_value=0).unstack(fill_value=0)
summary.columns = [' '.join(col).strip() for col in summary.columns.values]
summary

Day Weekday  Day Weekend  Night Weekday  \
Start Station ID End Station ID                                            
72               79                       84            7              6   
                 82                        1            0              0   
                 116                      75            9              3   
                 127                     197           58              5   
                 128                      50           36              2   
...                                      ...          ...            ...   
3908             3737                      1            1              0   
                 3740                      0            1              0   
                 3746                      7            0              0   
                 3749                      1            0              0   
                 3783                      6            0              0   

                                 Night Weekend  
Start Station ID End Station ID                 
72               79                          0  
                 82                          0  
                 116                         2  
                 127                         3  
                 128                         0  
...                                        ...  
3908             3737                        0  
                 3740                        0  
                 3746                        0  
                 3749                        0  
                 3783                        0  

[170427 rows x 4 columns]

In [21]:
summary['Day'] = summary['Day Weekday'] + summary['Day Weekend']
summary['Night'] = summary['Night Weekday'] + summary['Night Weekend']
summary['Weekday'] = summary['Night Weekday'] + summary['Day Weekday']
summary['Weekend'] = summary['Night Weekend'] + summary['Day Weekend']
summary['Total'] = summary['Day'] + summary['Night']
summary

Day Weekday  Day Weekend  Night Weekday  \
Start Station ID End Station ID                                            
72               79                       84            7              6   
                 82                        1            0              0   
                 116                      75            9              3   
                 127                     197           58              5   
                 128                      50           36              2   
...                                      ...          ...            ...   
3908             3737                      1            1              0   
                 3740                      0            1              0   
                 3746                      7            0              0   
                 3749                      1            0              0   
                 3783                      6            0              0   

                                 Night Weekend  Day  Night  Weekday  Weekend  \
Start Station ID End Station ID                                                
72               79                          0   91      6       90        7   
                 82                          0    1      0        1        0   
                 116                         2   84      5       78       11   
                 127                         3  255      8      202       61   
                 128                         0   86      2       52       36   
...                                        ...  ...    ...      ...      ...   
3908             3737                        0    2      0        1        1   
                 3740                        0    1      0        0        1   
                 3746                        0    7      0        7        0   
                 3749                        0    1      0        1        0   
                 3783                        0    6      0        6        0   

                                 Total  
Start Station ID End Station ID         
72               79                 97  
                 82                  1  
                 116                89  
                 127               263  
                 128                88  
...                                ...  
3908             3737                2  
                 3740                1  
                 3746                7  
                 3749                1  
                 3783                6  

[170427 rows x 9 columns]

In [22]:
summary = summary.reset_index()
summary

,Start Station ID,End Station ID,Day Weekday,Day Weekend,Night Weekday,Night Weekend,Day,Night,Weekday,Weekend,Total
0,72,79,84,7,6,0,91,6,90,7,97
1,72,82,1,0,0,0,1,0,1,0,1
2,72,116,75,9,3,2,84,5,78,11,89
3,72,127,197,58,5,3,255,8,202,61,263
4,72,128,50,36,2,0,86,2,52,36,88
...,...,...,...,...,...,...,...,...,...,...,...
170422,3908,3737,1,1,0,0,2,0,1,1,2
170423,3908,3740,0,1,0,0,1,0,0,1,1
170424,3908,3746,7,0,0,0,7,0,7,0,7
170425,3908,3749,1,0,0,0,1,0,1,0,1


In [23]:
for col in ['Day Weekday', 'Day Weekend', 'Night Weekday', 'Night Weekend', 'Day', 'Night', 'Weekday', 'Weekend', 'Total']:
    summary.sort_values(col, inplace=True, ascending=False)
    summary[f'{col}_R'] = range(len(summary))

In [24]:
ranks = [col for col in summary if col.endswith('_R')]
ranks

['Day Weekday_R',
 'Day Weekend_R',
 'Night Weekday_R',
 'Night Weekend_R',
 'Day_R',
 'Night_R',
 'Weekday_R',
 'Weekend_R',
 'Total_R']

In [26]:
# Only keep rides where one of the combinations ranks in 1000 most frequent
summary = summary[(summary[ranks] < 1000).any(axis=1)]
summary

,Start Station ID,End Station ID,Day Weekday,Day Weekend,Night Weekday,Night Weekend,Day,Night,Weekday,Weekend,Total,Day Weekday_R,Day Weekend_R,Night Weekday_R,Night Weekend_R,Day_R,Night_R,Weekday_R,Weekend_R,Total_R
47042,432,3263,6703,1075,562,86,7778,648,7265,1161,8426,0,17,16,77,0,13,0,17,0
83057,2006,3282,3658,2343,57,34,6001,91,3715,2377,6092,7,4,4515,1318,1,3186,11,4,1
155312,3664,327,5094,197,65,10,5291,75,5159,207,5366,1,1709,3624,11507,2,4549,1,1965,2
25920,327,3664,4145,241,872,16,4386,888,5017,257,5274,3,1066,0,5492,6,2,2,1199,3
73095,514,426,3245,1620,137,54,4865,191,3382,1674,5056,15,8,907,379,3,641,19,8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26072,328,411,21,1,41,44,22,85,62,45,107,71430,130691,7322,679,80553,3651,43767,21823,36516
163701,3734,174,20,10,29,46,30,75,49,56,105,74072,58134,11008,571,71191,4546,50652,17304,37006
138911,3491,519,3,0,60,39,3,99,63,39,102,127238,153596,4221,895,141147,2697,43353,24826,37839
109021,3259,2009,3,1,55,39,4,94,58,40,98,131081,121670,4755,890,127495,2983,45549,24319,38777


In [33]:
stations = stations[stations.Manhattan]
stations.id = stations.id.astype(int)
stations.set_index('id')
stations

,id,name,lon,lat,Manhattan
id,,,,,
3160,3160,Central Park West & W 76 St,-73.973747,40.778968,True
519,519,Pershing Square North,-73.977706,40.751873,True
3171,3171,Amsterdam Ave & W 82 St,-73.976673,40.785247,True
504,504,1 Ave & E 16 St,-73.981656,40.732219,True
229,229,Great Jones St,-73.993790,40.727434,True
...,...,...,...,...,...
3739,3739,Perry St & Greenwich Ave,-74.000939,40.735918,True
3731,3731,W 47 St & 6 Ave,-73.982550,40.758397,True
3744,3744,1 Ave & E 5 St,-73.986389,40.725864,True


In [34]:
summary.set_index('Start Station ID').join(stations)

,End Station ID,Day Weekday,Day Weekend,Night Weekday,Night Weekend,Day,Night,Weekday,Weekend,Total,...,Day_R,Night_R,Weekday_R,Weekend_R,Total_R,id,name,lon,lat,Manhattan
72,525,963,401,63,14,1364,77,1026,415,1441,...,550,4272,780,371,608,72,W 52 St & 11 Ave,-73.993929,40.767272,True
72,490,1184,145,66,17,1329,83,1250,162,1412,...,585,3817,489,3320,644,72,W 52 St & 11 Ave,-73.993929,40.767272,True
72,447,791,254,101,22,1045,123,892,276,1168,...,1002,1792,1053,1013,1003,72,W 52 St & 11 Ave,-73.993929,40.767272,True
72,514,819,246,43,17,1065,60,862,263,1125,...,955,6295,1145,1127,1093,72,W 52 St & 11 Ave,-73.993929,40.767272,True
72,426,676,282,23,10,958,33,699,292,991,...,1204,13516,1834,896,1430,72,W 52 St & 11 Ave,-73.993929,40.767272,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3746,368,595,208,79,38,803,117,674,246,920,...,1791,1983,1987,1349,1701,3746,6 Ave & Broome St,-74.004730,40.724308,True
3749,476,324,113,56,44,437,100,380,157,537,...,6122,2658,6109,3539,5312,3749,Lexington Ave & E 36 St,-73.978801,40.747574,True
3783,3664,148,40,146,6,188,152,294,46,340,...,19650,1160,9292,21133,11149,3783,Cliff St & Fulton St,-74.004950,40.708380,True
3788,3711,812,294,132,31,1106,163,944,325,1269,...,880,974,936,674,846,3788,E 12 St & 4 Av,-73.989740,40.732690,True


###  Export files to JSON

In [22]:
summary.to_json('paths.json', orient='index', indent=4)

In [26]:
stations.drop([0, 'Manhattan'], axis=1).set_index('start station id').to_json('stations.json', orient='records', indent=4)